In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

movies = pd.read_csv('data/movie.csv')
movies.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


# Agrupamiento, filtrado y transformación
Para hacer el análisis de datos tendremos que dividir nuestros datos en grupos para poder hacer cálculo sobre dichos
grupos. Los grupos se definirán mediante una o varias columnas que crearán un objeto para manejar el grupo.

La función que se usa para agrupar es **groupby**.

## Agregación

La primera tarea que se nos ocurre es la agregación.  Agragación es coombinar en un único resultado el grupo
correspondiente, crando tantos valores como grupos haya creados.

Por ejemplo, encontrar el máximo del mes, agrupando todos los datos en meses, por lo que aparecerán 12 valores.

La agregación utiliza dos componentes, el primero indicarán la/las columna/s que se van a agrupar para crear los
diferentes grupos. El segundo será la función que se usará con cada grupo para generar el valor buscado. Esta función
puede ser una predefinida (mean, max, min, etc) o crada por nosotros.

In [2]:
flights = pd.read_csv('data/flights.csv')
flights.groupby('AIRLINE')['ARR_DELAY'].agg('mean').head()

AIRLINE
AA    5.542661
AS   -0.833333
B6    8.692593
DL    0.339691
EV    7.034580
Name: ARR_DELAY, dtype: float64

In [3]:
flights.groupby('AIRLINE')['ARR_DELAY'].agg(np.mean).head()

AIRLINE
AA    5.542661
AS   -0.833333
B6    8.692593
DL    0.339691
EV    7.034580
Name: ARR_DELAY, dtype: float64

In [4]:
flights.groupby('AIRLINE')['ARR_DELAY'].mean().head()


AIRLINE
AA    5.542661
AS   -0.833333
B6    8.692593
DL    0.339691
EV    7.034580
Name: ARR_DELAY, dtype: float64

## Agrupamiento y agregación con varias columnas y funciones
Es posible hacer los grupos con varias columnas, siendo un grupo cada conjunto valores de todas las columnas único.

In [5]:
# Varias columnas
flights.groupby(['AIRLINE', 'WEEKDAY'])['CANCELLED'].sum()

AIRLINE  WEEKDAY
AA       1          41
         2           9
         3          16
         4          20
         5          18
                    ..
WN       3          18
         4          10
         5           7
         6          10
         7           7
Name: CANCELLED, Length: 98, dtype: int64

In [6]:
#Varias funciones, varias columnas
flights.groupby(['AIRLINE', 'WEEKDAY'])['CANCELLED'].agg(['sum', 'mean'])

sum      mean
AIRLINE WEEKDAY               
AA      1         41  0.032106
        2          9  0.007341
        3         16  0.011949
        4         20  0.015004
        5         18  0.014151
...              ...       ...
WN      3         18  0.014118
        4         10  0.007911
        5          7  0.005828
        6         10  0.010132
        7          7  0.006066

[98 rows x 2 columns]

In [7]:
#Varias funciones, varias columnas, diferentes funciones por columnas
group_cols = ['ORG_AIR', 'DEST_AIR']
agg_dict = {'CANCELLED':['sum', 'mean', 'size'],
            'AIR_TIME':['mean', 'var']}
flights.groupby(group_cols).agg(agg_dict)

CANCELLED                   AIR_TIME            
                       sum      mean size        mean         var
ORG_AIR DEST_AIR                                                 
ATL     ABE              0  0.000000   31   96.387097   45.778495
        ABQ              0  0.000000   16  170.500000   87.866667
        ABY              0  0.000000   19   28.578947    6.590643
        ACY              0  0.000000    6   91.333333   11.466667
        AEX              0  0.000000   40   78.725000   47.332692
...                    ...       ...  ...         ...         ...
SFO     SNA              4  0.032787  122   64.059322   11.338331
        STL              0  0.000000   20  198.900000  101.042105
        SUN              0  0.000000   10   78.000000   25.777778
        TUS              0  0.000000   20  100.200000   35.221053
        XNA              0  0.000000    2  173.500000    0.500000

[1130 rows x 5 columns]

## Eliminar el MultiIndex después de agrupar
Cuando se agrupa un Dataframe se crea un objeto MultiIndex para la gestión del mismo. Los Datasets con MultiIndex son
más difíciles de recorrer y tienen nombres de columnas y filas confusos.

In [8]:
group = flights.groupby(group_cols).agg(agg_dict)
group.head()

CANCELLED              AIR_TIME           
                       sum mean size        mean        var
ORG_AIR DEST_AIR                                           
ATL     ABE              0  0.0   31   96.387097  45.778495
        ABQ              0  0.0   16  170.500000  87.866667
        ABY              0  0.0   19   28.578947   6.590643
        ACY              0  0.0    6   91.333333  11.466667
        AEX              0  0.0   40   78.725000  47.332692

In [9]:
level0 = group.columns.get_level_values(0)
level1 = group.columns.get_level_values(1)
group.columns = level0 + '_' + level1
group.reset_index(inplace=True)
group.head()


,ORG_AIR,DEST_AIR,CANCELLED_sum,CANCELLED_mean,CANCELLED_size,AIR_TIME_mean,AIR_TIME_var
0,ATL,ABE,0,0.0,31,96.387097,45.778495
1,ATL,ABQ,0,0.0,16,170.500000,87.866667
2,ATL,ABY,0,0.0,19,28.578947,6.590643
3,ATL,ACY,0,0.0,6,91.333333,11.466667
4,ATL,AEX,0,0.0,40,78.725000,47.332692


## Funciones de agración propias
Podemos crear una función de agragación propia. Esta función debe recoger una Serie para realizar las operaciones
necesarias, generando un único valor.

In [10]:
def max_deviation(s):
    std_score = (s - s.mean()) / s.std()
    return std_score.abs().max()
flights.groupby('ORG_AIR')['AIR_TIME'].agg(max_deviation).round(1).head()


ORG_AIR
ATL    8.5
DEN    7.5
DFW    6.8
IAH    6.6
LAS    3.4
Name: AIR_TIME, dtype: float64

## Propiedades del objeto groupby

In [11]:
group = flights.groupby(group_cols)
group.ngroups

1130

In [12]:
list(group.groups.keys())[:6]


[('ATL', 'ABE'),
 ('ATL', 'ABQ'),
 ('ATL', 'ABY'),
 ('ATL', 'ACY'),
 ('ATL', 'AEX'),
 ('ATL', 'AGS')]

## Grouping by continuous variables
Qué pasa cuando en los valores de una columna no hay repetidos. Para esos casos podemos usar la función **cut** para
"discretizar" los valores de una columna.

In [13]:
flights = pd.read_csv('data/flights.csv')
flights.dropna(subset=['DIST'], inplace=True)
bins = [-np.inf, 200, 500, 1000, 2000, np.inf]
cuts = pd.cut(flights['DIST'], bins=bins)
cuts.head()

0     (500.0, 1000.0]
1    (1000.0, 2000.0]
2     (500.0, 1000.0]
3    (1000.0, 2000.0]
4    (1000.0, 2000.0]
Name: DIST, dtype: category
Categories (5, interval[float64]): [(-inf, 200.0] < (200.0, 500.0] < (500.0, 1000.0] < (1000.0, 2000.0] < (2000.0, inf]]

In [14]:
cuts.value_counts()

(500.0, 1000.0]     20659
(200.0, 500.0]      15874
(1000.0, 2000.0]    14186
(2000.0, inf]        4054
(-inf, 200.0]        3719
Name: DIST, dtype: int64

In [15]:
flights.groupby(cuts)['AIRLINE'].value_counts(normalize=True).round(3)




DIST              AIRLINE
(-inf, 200.0]     OO         0.326
                  EV         0.289
                  MQ         0.211
                  DL         0.086
                  AA         0.052
                  UA         0.027
                  WN         0.009
(200.0, 500.0]    WN         0.194
                  DL         0.189
                  OO         0.159
                  EV         0.156
                  MQ         0.100
                  AA         0.071
                  UA         0.062
                  VX         0.028
                  US         0.016
                  NK         0.012
                  B6         0.007
                  F9         0.005
                  AS         0.001
(500.0, 1000.0]   DL         0.206
                  AA         0.144
                  WN         0.138
                  UA         0.131
                  OO         0.106
                  EV         0.101
                  MQ         0.051
                  F9         